# This notebook is used to extract fixations for each participant

In [2]:
from deepeye_analysis_package.getFixations import batch_extract_fixations

path_to_data = r'D:/Dropbox/Appliedwork/CognitiveSolutions/Projects/DeepEye/TechnicalReports/TechnicalReport1/Test_PreferentialViewing/Pilot_PreferentialViewing/withDelay/Approved/data'
batch_extract_fixations(path_to_data)

Processing participant 2024_09_25_11_53_02...


d:\GitHub\DeepEye_analyze\deepeye_analysis_package\getFixations.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['frameNr'] = pd.to_numeric(df['frameNr'], errors='coerce')  # Convert frameNr to numeric, set non-numeric to NaN





Importing and processing: "D:/Dropbox/Appliedwork/CognitiveSolutions/Projects/DeepEye/TechnicalReports/TechnicalReport1/Test_PreferentialViewing/Pilot_PreferentialViewing/withDelay/Approved/data\2024_09_25_11_53_02\2024_09_25_11_53_02_record.csv"
	Searching for valid interpolation windows
	Replace interpolation windows with Steffen interpolation
	2-Means clustering started for averaged signal
	Determining fixations based on clustering weight mean for averaged signal and separate eyes + 2*std


I2MC took 5.721678256988525s to finish!
Processing complete.
